In [227]:
# imports
import pandas as pd
import os
import numpy as np

In [231]:
import pickle

In [232]:
file_name = "diverse_data"
fileObject = open(file_name, 'rb')
diverse_data = pickle.load(fileObject)
fileObject.close()

In [241]:
book_table.index

Int64Index([306, 790, 115], dtype='int64')

In [246]:
115 in diverse_data.index.difference(book_table.index)

False

In [248]:
diverse_data.iloc[115]

ISBN                                                      1466816139
author                                                 Walter Mosley
avg_ratings                                                     3.35
awards                                                            []
character_names                                                   []
description        [In ancient mythology, the Titan Prometheus wa...
genres                        [Fiction, Fantasy, Mystery, Audiobook]
language                                                     English
num_pages                                                        288
num_ratings                                                      140
num_reviews                                                       19
places                                                            []
publish_date                                     2012-05-08 00:00:00
title              The Gift of Fire: A Novel from Crosstown to Ob...
url                https://www.goo

In [250]:
dtable.iloc[115]

ISBN                                                      1932841725
author                                                  Kiese Laymon
avg_ratings                                                     3.78
awards             [William Saroyan International Prize for Writi...
character_names                                                   []
description        [Kiese Laymon’s debut novel is a Twain-esque e...
genres             [Fiction, Science Fiction, Time Travel, Scienc...
language                                                     English
num_pages                                                        276
num_ratings                                                    2,184
num_reviews                                                      414
places                                                            []
publish_date                                     2013-06-11 00:00:00
title                                                  Long Division
url                https://www.goo

In [240]:
dtable = diverse_data.iloc[diverse_data.index.difference(book_table.index)]

In [237]:
book_table = diverse_data.sample(3)

In [68]:
from collections import defaultdict

In [90]:
# read in data
# some of these lists are weird...and maybe shouldn't be used...
# or at least filtered by review and maybe genre to eliminate weird entries

root = './data/'
files = os.listdir(root)
dframes = [pd.read_json(root + f) for f in files]
#data = pd.read_json(fname)
#data

# The descriptions are screwed up - parts of them repeat.  I'm not sure what to do about this.  
# If first reducing to words, could live with it and use unique words.

In [91]:
genres = defaultdict(int)
for dframe in dframes:
    for l in dframe.genres:
        for li in l:
            genres[li] += 1

In [96]:
sorted([(value,key) for key,value in genres.items()])[::-1]

[(7633, 'Fiction'),
 (4810, 'Historical'),
 (3962, 'Cultural'),
 (3140, 'Historical Fiction'),
 (2617, 'Classics'),
 (2406, 'Literature'),
 (2271, 'Fantasy'),
 (2262, 'Young Adult'),
 (1524, 'Romance'),
 (1482, 'Mystery'),
 (1365, 'European Literature'),
 (1240, 'War'),
 (1161, 'Nonfiction'),
 (1112, 'Science Fiction'),
 (1109, 'Contemporary'),
 (867, 'Childrens'),
 (780, 'Novels'),
 (755, 'Short Stories'),
 (702, 'World War II'),
 (670, 'History'),
 (612, 'Africa'),
 (541, 'Canada'),
 (530, 'Biography'),
 (527, 'Autobiography'),
 (520, 'France'),
 (480, 'Memoir'),
 (471, 'Thriller'),
 (439, 'Humor'),
 (416, 'Adventure'),
 (398, 'Crime'),
 (394, 'Paranormal'),
 (365, 'Poetry'),
 (351, 'Literary Fiction'),
 (348, 'British Literature'),
 (343, 'Academic'),
 (324, 'Russia'),
 (317, 'Travel'),
 (298, 'Asian Literature'),
 (294, 'Horror'),
 (292, 'Lgbt'),
 (290, 'Realistic Fiction'),
 (275, 'Dystopia'),
 (272, 'Asia'),
 (254, 'School'),
 (248, 'Plays'),
 (235, 'India'),
 (230, 'Magical Real

In [71]:
master = pd.concat(dframes)

In [72]:
np.sum(master.author.str.contains('kipling'))

0

In [204]:
from validation import top_books

In [205]:
best_books = pd.DataFrame(top_books.top_books)

In [196]:
from search_book_name import *
desc_input = search_id(search_name('The Hobbit'))

In [226]:
#desc_input

In [192]:
def search_name(name):
    #name = 'The Hounds of Baskerville'#254656
    name_split = name.split()
    name_join = '+'.join(name_split)
    url_name = 'https://www.goodreads.com/search/index.xml?key=sydsjovGS88bdTfMwAI2Ug&q=' + name_join
    
    name_search = requests.get(url_name)
    soup_tmp = bs(name_search.content, features='xml')#parse file
    gr_id = soup_tmp.findAll('id')[1].get_text()
    return gr_id

def search_id(gr_id):
    url_id = 'https://www.goodreads.com/book/show/' + str(gr_id) + '.xml?key=sydsjovGS88bdTfMwAI2Ug'
    id_search = requests.get(url_id)

    id_soup = bs(id_search.content, features='xml')
    book_genre = id_soup.findAll('genre')[0].get_text()
    return id_soup#book_desc,book_genre

def search_genres_id(gr_id):
    url_id = 'https://www.goodreads.com/book/show/' + str(gr_id) + '.xml?key=sydsjovGS88bdTfMwAI2Ug'
    id_search = requests.get(url_id)
    id_soup = bs(id_search.content, features='xml')
    shlfs = id_soup.findAll('popular_shelves')[0].findAll('shelf')
    book_genres = [(s.attrs['count'],s.attrs['name']) for s in shlfs]
    return book_genres

In [199]:
desc_input = search_id(search_name('The Hobbit'))

In [172]:
shlfs = desc_input.findAll('popular_shelves')[0].findAll('shelf')
book_genres = set([s.attrs['name'] for s in shlfs])

In [200]:
book_genres = search_genres_id(search_name('First Day Jitters'))

In [211]:
titles = [title for title in best_books['title']]

In [220]:
book_genres = dict()
for t in titles:
    try:
        book_genres[t] = search_genres_id(search_name(t))
    except IndexError:
        print(t)

First Day Jitters (Mrs. Hartwells classroom adventures)
Multiplication 0 to 12 Flash Cards (Brighter Child Flash Cards)
Potty (Leslie Patricelli board books)
My Fox Ate My Homework (a hilarious fantasy for children ages 8-12)
School Zone - Big Preschool Workbook - Ages 4 and Up, Colors, Shapes, Numbers 1-10, Alphabet, Pre-Writing, Pre-Reading, and author:Phonics (Big Get Ready Workbook)
Preschool Scholar Deluxe Edition Workbook, Ages 3-5, tracing letters & numbers, learning shapes & colors, animal names, playful motivation
Division 0 to 12 Flash Cards (Brighter Child Flash Cards)
Chicka Chicka Boom Boom (Board Book)
Klutz LEGO Chain Reactions Craft Kit


In [224]:
book_genres_tokens = dict()
for key,value in book_genres.items():
    if len(value)>1:
        book_genres_tokens[key] = ' '.join([x[1] for x in value])

In [225]:
book_genres_tokens

{'The Little Prince': 'to-read favorites currently-reading classics fiction fantasy childrens children classic owned french books-i-own favourites children-s young-adult philosophy children-s-books childhood kids literature to-read owned-books childrens-books french-literature school france 1001-books novels re-read favorite ya novel to-buy default middle-grade children-s-literature adventure my-books all-time-favorites children-s-lit kids-books library 1001 favorite-books my-library favoritos 20th-century français books read-in-2016 i-own 5-stars juvenile children-books inspirational read-in-2015 read-for-school read-in-2017 favourite translated childhood-favorites to-re-read picture-books read-in-french ebook french-lit childrens-lit short-stories reread kindle classic-literature faves childrens-literature childhood-books science-fiction clásicos high-school must-read book-club stand-alone for-school 1001-books-to-read-before-you-die english read-more-than-once favs to-reread borrowe